# Using Python to Access Your E-mail

Python has several modules in its standard library to work with e-mail and other web tools. Using Python you can compose and send e-mails as well as retrieve e-mails from mail servers and parse the content of e-mail files. In this notebook we will do the later two.

For the example here I am going to use my University of Utah e-mail. Similar approaches can be used for other e-mail providers. For example, [here](https://developers.google.com/gmail/api/quickstart/python) are instructions from Google about how to interact with Gmail via Python.

In [1]:
import os
DATADIR = os.path.join(os.getcwd())
import csv
import imaplib
import getpass
import email
from collections import defaultdict
import gzip
import pickle


# Working with E-Mail
* Python has several modules for working with e-mail, including sending e-mails (not goint o talk about this), working with an inbox, and parsing e-mail messages
* [imaplib](http://docs.python.org/2/library/imaplib.html)
* Below is a code snippet adapted from the Python documentation
* Some notes:
    * **``getpass.getpass()``** prompts for a password without echoing it back to the screen
    * also **``getpass.getuser()``**; 

### Here is a script to connect to and pull e-mails from UMail

This was very slow for me, probably largely because I don't delete enough e-mails.

* [``imaplib.IMAP4_SSl``](https://docs.python.org/3/library/imaplib.html): "This is a subclass derived from IMAP4 that connects over an SSL encrypted socket."
* [``getpass.getpass``](https://docs.python.org/3.5/library/getpass.html) ``getpass`` allows us to get passwords (or other text) that we don't want echoed back to the screen. As best practice pass the results of getpass directly to the function that needs the password so that you don't have a variable floating around withe sensitive information.

In [4]:

M = imaplib.IMAP4_SSL("imap.umail.utah.edu",port=993)
M.login('%s@umail.utah.edu'%getpass.getpass("Enter your University of Utah ID").strip(),
        getpass.getpass("Enter your University of Utah password").strip())
M.select()
typ,data = M.search(None,"ALL")
msgs = {}
count = 0
for num in data[0].split():
    count += 1
    # My inbox had around 12000 messages in it. 
    # this was a way to keep my up to date on whether
    # my program was really progressing
    if count %500 == 0:
        print (num),
    typ,data = M.fetch(num,'(RFC822)')
    msgs[num] = data


Enter your University of Utah ID········
Enter your University of Utah password········
b'500'


### Write everything out to a pickle file

* I don't want to query my e-mail very often since it is so slow, so let's save the data for later use.

In [5]:
with gzip.open(os.path.join(DATADIR,
               "myEmail11192017.pickle.gzip"),"wb") as fo:
    pickle.dump(msgs,fo)

#### If we want to start over we can just read in the pickle file and skip the IMAP step

In [6]:
with gzip.open(os.path.join(DATADIR,
               "myEmail11192017.pickle.gzip"),"rb") as fo:
    msgs = pickle.load(fo)

# Parsing e-mail messages
* [email:](https://docs.python.org/3/library/email.html#module-email)

>The email package is a library for managing email messages, including MIME and other RFC 2822-based message documents. It is specifically not designed to do any sending of email messages to SMTP (RFC 2821), NNTP, or other servers; those are functions of modules such as smtplib and nntplib. The email package attempts to be as RFC-compliant as possible, supporting in addition to RFC 2822, such MIME-related RFCs as RFC 2045, RFC 2046, RFC 2047, and RFC 2231. (Python Documentation)

## Read e-mails and save 'From'/'to' and `date` information
### Always some unicode confusion



### What does a message look like?

In [9]:
m = msgs[b'500'] #500 is the key
type(m), len(m)

(list, 2)

In [10]:
print(m)

[(b'500 (RFC822 {23005}', b'Received: from X-MB7.xds.umail.utah.edu ([169.254.12.138]) by\r\n X-HUB3.xds.umail.utah.edu ([155.97.181.103]) with mapi id 14.03.0361.001;\r\n Thu, 12 Oct 2017 13:59:54 -0600\r\nFrom: "Megan E. Williams" <megan.williams@neuro.utah.edu>\r\nTo: KATHLEEN JENNIFER BENNETT <kathleen.bennett@utah.edu>\r\nSubject: Re: TA dev Neuro\r\nThread-Topic: TA dev Neuro\r\nThread-Index: AQHTJyZebYL8newNIEqzRaco76H78aKow0+AgDfsaACAAHX/gP//tVQA\r\nDate: Thu, 12 Oct 2017 13:59:54 -0600\r\nMessage-ID: <4A9BA5E5-903B-4923-A859-E5F8271D9273@neuro.utah.edu>\r\nReferences: <61B2DBCF-4E88-4D67-A0F7-87B9E20C8912@neuro.utah.edu>\r\n <CAEVaRPZ0yHyhJ8DncHHUQHyDCZeeuJDPAqdVYt9t3WXx+ueAzw@mail.gmail.com>\r\n <C7500B05-7A17-41F1-B579-985AB993414C@neuro.utah.edu>\r\n <CAEVaRPYzcM3PgGp9LF_SYGWkVcii3FihwnR_Q1giVs2__DC8ZQ@mail.gmail.com>\r\nIn-Reply-To:\r\n <CAEVaRPYzcM3PgGp9LF_SYGWkVcii3FihwnR_Q1giVs2__DC8ZQ@mail.gmail.com>\r\nAccept-Language: en-US\r\nContent-Language: en-US\r\nX-MS-Exchange

#### The message is a two-tuple
* The first element is another tuple 
    * The first element of which is some index information.
    * The second element is a big, nasty string.
* The second element is a string describing the message status

In [11]:
print("The length of the message tuple is %s"%len(m))
print(m[0][0],m[1])


The length of the message tuple is 2
b'500 (RFC822 {23005}' b' FLAGS (\\Seen))'


In [12]:
print(m[0][1].decode()[0:350])


Received: from X-MB7.xds.umail.utah.edu ([169.254.12.138]) by
 X-HUB3.xds.umail.utah.edu ([155.97.181.103]) with mapi id 14.03.0361.001;
 Thu, 12 Oct 2017 13:59:54 -0600
From: "Megan E. Williams" <megan.williams@neuro.utah.edu>
To: KATHLEEN JENNIFER BENNETT <kathleen.bennett@utah.edu>
Subject: Re: TA dev Neuro
Thread-Topic: TA dev Neuro
Thre


### Now we need to Parse the messages

* Create an e-mail parser
* Take a look at what a parsed message looks like

* [parsestr](https://docs.python.org/3/library/email.parser.html#email.parser.Parser.parsestr)

* Lots and lots of header information
* Text of e-mail is buried in a bunch of HTML that would have to be parsed.

In [13]:
p = email.parser.Parser()
e = p.parsestr(m[0][1].decode())
print(e.keys())


['Received', 'From', 'To', 'Subject', 'Thread-Topic', 'Thread-Index', 'Date', 'Message-ID', 'References', 'In-Reply-To', 'Accept-Language', 'Content-Language', 'X-MS-Exchange-Organization-AuthAs', 'X-MS-Exchange-Organization-AuthMechanism', 'X-MS-Exchange-Organization-AuthSource', 'X-MS-Has-Attach', 'X-MS-Exchange-Organization-SCL', 'X-MS-Exchange-Inbox-Rules-Loop', 'X-MS-TNEF-Correlator', 'Content-Type', 'Content-ID', 'Content-Transfer-Encoding', 'MIME-Version']


In [14]:
for k,v in e.items():
    print(k)
    print(v)
    print()

Received
from X-MB7.xds.umail.utah.edu ([169.254.12.138]) by
 X-HUB3.xds.umail.utah.edu ([155.97.181.103]) with mapi id 14.03.0361.001;
 Thu, 12 Oct 2017 13:59:54 -0600

From
"Megan E. Williams" <megan.williams@neuro.utah.edu>

To
KATHLEEN JENNIFER BENNETT <kathleen.bennett@utah.edu>

Subject
Re: TA dev Neuro

Thread-Topic
TA dev Neuro

Thread-Index
AQHTJyZebYL8newNIEqzRaco76H78aKow0+AgDfsaACAAHX/gP//tVQA

Date
Thu, 12 Oct 2017 13:59:54 -0600

Message-ID
<4A9BA5E5-903B-4923-A859-E5F8271D9273@neuro.utah.edu>

References
<61B2DBCF-4E88-4D67-A0F7-87B9E20C8912@neuro.utah.edu>
 <CAEVaRPZ0yHyhJ8DncHHUQHyDCZeeuJDPAqdVYt9t3WXx+ueAzw@mail.gmail.com>
 <C7500B05-7A17-41F1-B579-985AB993414C@neuro.utah.edu>
 <CAEVaRPYzcM3PgGp9LF_SYGWkVcii3FihwnR_Q1giVs2__DC8ZQ@mail.gmail.com>

In-Reply-To

 <CAEVaRPYzcM3PgGp9LF_SYGWkVcii3FihwnR_Q1giVs2__DC8ZQ@mail.gmail.com>

Accept-Language
en-US

Content-Language
en-US

X-MS-Exchange-Organization-AuthAs
Internal

X-MS-Exchange-Organization-AuthMechanism
04

X-MS-

In [15]:
e["date"]

'Thu, 12 Oct 2017 13:59:54 -0600'

In [16]:
import re
from itertools import product
rclean = re.compile(r"""\s+""")
remail = re.compile(r"""<(?P<email>\S+@\S+)>""")

#### How do we want to simplify our data?

* No consistency in how names are provided (e.g. "Yiling Bi" or "Bi, Yiling")
* `From` is from one person
* `To` can be to one to many people
    * Sometimes I find blank entries for both "To" and "From"
    
#### In the cell below I'm doing the following:

* I'm only keeping "To", "From", and "date" information
* I'm going to identify each recipient in the "To" list using a regular expression and make a node/edge relationship for each person in the "To" list.
* Write these out into a tab delimited file


In [17]:

with open(os.path.join(DATADIR,
            "my_emails_2017.txt"),"wt") as fo:
    for key in msgs.keys():
        m = msgs[key]
        try:
            e = p.parsestr(m[0][1].decode())
        except UnicodeDecodeError:
            e = p.parsestr(m[0][1].decode('windows-1252'))
        if e["To"] and e["From"]:
            for f,t in product(remail.findall(e["From"]), remail.findall(e["To"])):
                fo.write("%s\t%s\t%s\n"%(f, t, e["date"]))